In [1]:
# IMPORTS
import altair as alt
import pandas as pd
from vega_datasets import data
alt.data_transformers.disable_max_rows()

food_emissions = pd.read_excel("/Users/aksharkatariya/Documents/GitHub/static_project/EDGAR-FOOD_v6.xlsx", sheet_name="emissions", header=3)
shares = pd.read_excel("/Users/aksharkatariya/Documents/GitHub/static_project/EDGAR-FOOD_v6.xlsx", sheet_name="shares", header=3)
sector = pd.read_excel("/Users/aksharkatariya/Documents/GitHub/static_project/EDGAR-FOOD_v6.xlsx", sheet_name="sector", header=3)
country_definition = pd.read_excel("/Users/aksharkatariya/Documents/GitHub/static_project/EDGAR-FOOD_v6.xlsx", sheet_name="country_definition", header=3)
df_population = pd.read_csv('/Users/aksharkatariya/Downloads/population.csv')
gdp_data = pd.read_csv('/Users/aksharkatariya/Downloads/national-gdp-constant-usd-wb.csv')



In [2]:
country_definition.head()
country_definition.rename(columns={'Country ISO code': 'Code'})

,Code,Country name,Regional grouping,Country class
0,ABW,Aruba,Rest Central America,Developing
1,AFG,Afghanistan,India +,Developing
2,AGO,Angola,Southern_Africa,Developing
3,AIA,Anguilla,Rest Central America,Developing
4,AIR,Int. Aviation,Int. Aviation,0
...,...,...,...,...
219,WSM,Samoa,Oceania,Developing
220,YEM,Yemen,Middle_East,Developing
221,ZAF,South Africa,Southern_Africa,Developing
222,ZMB,Zambia,Southern_Africa,Developing


In [3]:
sector.head()

,Country_code_A3,Name,C_group_IM24_sh,dev_country,Substance,FOOD_system_stage,Y_1990,Y_1991,Y_1992,Y_1993,...,Y_2009,Y_2010,Y_2011,Y_2012,Y_2013,Y_2014,Y_2015,Y_2016,Y_2017,Y_2018
0,AGO,Angola,10:_Southern_Africa,D,GWP_100_CO2,LULUC (Production),10273.5451,10273.5451,10273.5451,10273.5451,...,35606.0653,35605.4601,35604.1556,35604.1556,35604.1556,35602.6343,35601.0571,35597.6207,35597.6207,35592.0828
1,AGO,Angola,10:_Southern_Africa,D,GWP_100_CH4,LULUC (Production),7662.4856,7662.4856,7662.4856,7662.4856,...,6739.5328,7193.0068,6248.5080,6645.6348,5922.2156,6228.5104,6667.6736,6293.3276,7681.6264,4870.0764
2,AGO,Angola,10:_Southern_Africa,D,GWP_100_N2O,LULUC (Production),5009.7985,5009.7985,5009.7985,5009.7985,...,5869.6705,6261.4995,5445.1405,5788.2890,5163.1805,5427.7300,5807.0510,5483.2475,6682.9290,4252.8790
3,ALB,Albania,12:_Central Europe,I,GWP_100_CO2,LULUC (Production),116.8068,116.8068,116.8068,116.8068,...,116.6195,116.6195,116.6195,116.6195,116.5582,116.5582,116.4970,116.3398,116.3398,114.1346
4,ALB,Albania,12:_Central Europe,I,GWP_100_N2O,LULUC (Production),12.4815,12.4815,12.4815,12.4815,...,12.4550,12.4550,12.4550,12.4550,12.4550,12.4550,12.4550,12.4020,12.4020,12.1900


In [4]:
sector.head()
sector['C_group_IM24_sh'] = sector['C_group_IM24_sh'].str[4:]
sector['Substance'] = sector['Substance'].str[8:]
sector.head()
sector_grouped = sector.groupby('Substance').agg({col: 'mean' for col in sector.columns if col.startswith('Y_') and int(col[2:]) in range(1990, 2019)})
sector_grouped.reset_index(inplace=True)
sector['dev_country'] = sector['dev_country'].replace({'I': 'Industrialized', 'D': 'Developing'})


In [5]:
tidy_sector_grouped = pd.melt(sector_grouped, id_vars=['Substance'], 
                  value_vars=[col for col in sector_grouped.columns if col.startswith('Y_')],
                  var_name='Year', value_name='Emissions')


tidy_sector_grouped['Year'] = tidy_sector_grouped['Year'].str[2:].astype(int)

In [6]:
alt.Chart(tidy_sector_grouped).mark_area(strokeWidth=4).encode(
    x=alt.X("Year:O", title='Year'),
    y=alt.Y("Emissions", axis=alt.Axis(title="Emissions (Kilotons)", labels=True), scale=alt.Scale(zero=False)),
    color=alt.Color(
        "Substance",
        scale=alt.Scale(domain=["CH4", "CO2", "F-gases", "N2O"],  # Replace with actual substance names
                        range=['#51d7ca', '#a0a6e9','#283677' ,'#1b595b' ]),
        legend=alt.Legend(title=None,
                         labelFontSize = 22)
    )
).properties(
    title=alt.TitleParams(
        text='Greenhouse Gas Emissions Over 3 Decades',
        subtitle='F-gases (fluorinated gases) emissions have increased 7x in the last 20 years',
        fontSize=24,
        font='Arial',
        color='Black',
        subtitleFontSize=18,
        subtitleFontStyle='italic',
        subtitleColor='darkgray'
    )
).configure_title(
    anchor='start'
)

/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [7]:

sector_grouped1 = sector.groupby(['Substance', 'FOOD_system_stage']).agg({col: 'mean' for col in sector.columns if col.startswith('Y_') and int(col[2:]) in range(1990, 2019)}).reset_index()

tidy_sector_grouped1 = pd.melt(sector_grouped1, id_vars=['Substance', 'FOOD_system_stage'], 
                  value_vars=[col for col in sector_grouped.columns if col.startswith('Y_')],
                  var_name='Year', value_name='Emissions')

tidy_sector_grouped1['Year'] = tidy_sector_grouped1['Year'].str[2:].astype(int)
tidy_sector_grouped1

,Substance,FOOD_system_stage,Year,Emissions
0,CH4,Consumption,1990,451.590964
1,CH4,End_of_Life,1990,4834.805043
2,CH4,LULUC (Production),1990,1603.945560
3,CH4,Packaging,1990,176.458417
4,CH4,Processing,1990,381.748408
...,...,...,...,...
749,N2O,Packaging,2018,22.291563
750,N2O,Processing,2018,122.419674
751,N2O,Production,2018,8497.065728
752,N2O,Retail,2018,5.437893


In [8]:

tidy_sector_grouped1['FOOD_system_stage'] = tidy_sector_grouped1['FOOD_system_stage'].replace({
    'LULUC (Production)': 'Land use and land-use changes',
    'End_of_Life': 'End of Life'
})


chart = alt.Chart(tidy_sector_grouped1).mark_area().encode(
    x=alt.X('Year:O', title=None, axis=alt.Axis(labelExpr="datum.value % 2 === 0 ? datum.value : ''")),  # Show labels only for every five years
    y=alt.Y('Emissions:Q', stack='normalize', title=None),  
    color=alt.Color(
        "Substance",
        scale=alt.Scale(domain=["CO2", "CH4", "N2O", "F-gases"],  
                        range=['#a0a6e9', '#51d7ca', '#1b595b', '#283677']),
        legend=None  
    ),
    facet=alt.Facet('FOOD_system_stage:N', columns=2, spacing={"row": 20, "column": 105}, title =None, header=alt.Header(labelFontSize=18))  # Adjust columns as needed
).properties(
    title=alt.TitleParams(
        text='Greenhouse Gas Emissions by Food System Stage (Normalized)',
        subtitle='Retial in the food cycle stages is the only one which contributes to F-gases',
        fontSize=22,
        font='Arial',
        color='Black',
        subtitleFontSize=16,
        subtitleFontStyle='italic',
        subtitleColor='darkgray'
    ),
        width = 220, 
        height = 160
)

chart

/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [9]:
gdp_data.head()

,Entity,Code,Year,GDP (constant 2015 US$)
0,Afghanistan,AFG,2002,7228796000
1,Afghanistan,AFG,2003,7867263500
2,Afghanistan,AFG,2004,7978515500
3,Afghanistan,AFG,2005,8874481000
4,Afghanistan,AFG,2006,9349922000


In [10]:
# Filter for rows where Year > 1990
gdp_data = gdp_data[gdp_data['Year'] >= 1990]
gdp_data.head()


,Entity,Code,Year,GDP (constant 2015 US$)
0,Afghanistan,AFG,2002,7228796000
1,Afghanistan,AFG,2003,7867263500
2,Afghanistan,AFG,2004,7978515500
3,Afghanistan,AFG,2005,8874481000
4,Afghanistan,AFG,2006,9349922000


In [11]:
sector = sector.rename(columns = {'Country_code_A3' : 'Code'})

In [12]:

sector_grouped = sector.groupby(['Substance', 'FOOD_system_stage', 'Code']).agg({col: 'mean' for col in sector.columns if col.startswith('Y_') and int(col[2:]) in range(1990, 2019)}).reset_index()
sector_grouped.head()
tidy_sector_grouped = pd.melt(sector_grouped, id_vars=['Substance', 'FOOD_system_stage', 'Code'], 
                  value_vars=[col for col in sector_grouped.columns if col.startswith('Y_')],
                  var_name='Year', value_name='Emissions')

tidy_sector_grouped['Year'] = tidy_sector_grouped['Year'].str[2:].astype(int)
tidy_sector_grouped

,Substance,FOOD_system_stage,Code,Year,Emissions
0,CH4,Consumption,ABW,1990,0.018076
1,CH4,Consumption,AFG,1990,6.145408
2,CH4,Consumption,AGO,1990,274.731055
3,CH4,Consumption,AIA,1990,0.000003
4,CH4,Consumption,ALB,1990,0.993114
...,...,...,...,...,...
140500,N2O,Transport,WSM,2018,0.188467
140501,N2O,Transport,YEM,2018,2.381084
140502,N2O,Transport,ZAF,2018,72.891769
140503,N2O,Transport,ZMB,2018,1.587201


In [13]:
merged_data = pd.merge(gdp_data, tidy_sector_grouped, on=['Code', 'Year'], how='inner')
merged_data.tail()

,Entity,Code,Year,GDP (constant 2015 US$),Substance,FOOD_system_stage,Emissions
122388,Zimbabwe,ZWE,2018,22015179000,N2O,Packaging,0.775733
122389,Zimbabwe,ZWE,2018,22015179000,N2O,Processing,29.990585
122390,Zimbabwe,ZWE,2018,22015179000,N2O,Production,3572.479803
122391,Zimbabwe,ZWE,2018,22015179000,N2O,Retail,0.185402
122392,Zimbabwe,ZWE,2018,22015179000,N2O,Transport,4.331293


## Country GDP Merge

In [14]:
food_emissions

,Country_code_A3,Name,Y_1990,Y_1991,Y_1992,Y_1993,Y_1994,Y_1995,Y_1996,Y_1997,...,Y_2009,Y_2010,Y_2011,Y_2012,Y_2013,Y_2014,Y_2015,Y_2016,Y_2017,Y_2018
0,ABW,Aruba,52.467587,58.445779,66.702558,62.301504,68.506353,75.829265,58.734385,81.162564,...,159.364420,141.075838,135.129301,173.603909,169.526161,172.058516,169.902421,172.877401,158.317865,154.667162
1,AFG,Afghanistan,10625.107867,10830.881104,10950.536732,11171.821501,11299.594874,11815.267602,13155.159112,14252.954840,...,18810.398534,20963.797333,21558.233158,21216.818799,20942.206787,21451.730195,21147.538782,21050.778267,21302.708366,21196.132147
2,AGO,Angola,32156.596760,32464.235278,32567.388288,32602.146277,34115.353337,34801.965224,40930.126655,40057.615721,...,65693.104142,67693.467207,66277.366276,67652.087410,67200.277612,68146.581224,69470.341229,69064.557603,71539.342723,66445.751315
3,AIA,Anguilla,2.726608,3.009267,3.130169,3.406827,3.932457,4.488219,4.493260,4.194689,...,5.795507,6.373054,6.660545,6.705420,6.993232,7.027910,7.078796,7.159170,7.272695,7.181544
4,AIR,Int. Aviation,1322.248427,1285.268168,1325.968003,1352.131430,1424.838702,1481.416851,1541.112346,1620.190699,...,2216.413310,2339.399901,2426.222186,2445.366423,2487.015536,2570.114990,2712.218964,2831.883565,2990.773933,3116.257046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,WSM,Samoa,290.512354,289.997516,285.387323,289.780475,291.786046,288.328443,288.244241,289.632853,...,311.323209,323.402693,348.916498,343.480279,347.314610,353.513520,376.688599,359.397828,370.633245,372.112812
219,YEM,Yemen,7794.380638,7908.602165,8182.372939,8423.801754,8887.452732,9470.029094,10014.666291,10670.328643,...,19073.221739,19282.736617,18811.873534,18428.604889,20329.478240,20610.039484,17540.069348,17252.588662,17614.995147,18560.161634
220,ZAF,South Africa,77801.966152,77864.928809,77285.098143,76910.205925,78758.737855,80443.466647,81997.755870,83449.988954,...,103471.998694,102818.702147,104366.359233,105817.538784,107756.800263,108612.774826,107605.876048,107160.431744,111095.178574,110973.875397
221,ZMB,Zambia,42792.349176,43022.199380,43007.008759,42850.828425,42472.604034,42129.659454,36655.298480,37255.153033,...,44450.086045,48009.862215,62564.699629,64659.497155,65265.030857,65948.220750,69639.303501,68264.057467,69489.261647,67302.877038


In [15]:
food_emissions = food_emissions.rename(columns = {'Country_code_A3' : 'Code',
                                                 'Name' : 'Entity'})
food_emissions = pd.melt(food_emissions, id_vars=['Code', 'Entity'], 
                  value_vars=[col for col in sector_grouped.columns if col.startswith('Y_')],
                  var_name='Year', value_name='Food Emissions')

food_emissions['Year'] = food_emissions['Year'].str[2:].astype(int)


In [16]:
merged_data = pd.merge(food_emissions, gdp_data, on=['Code', 'Entity', 'Year'], how='inner')
merged_data.head()

,Code,Entity,Year,Food Emissions,GDP (constant 2015 US$)
0,ABW,Aruba,1990,52.467587,1669818600
1,AGO,Angola,1990,32156.596760,27672785000
2,ALB,Albania,1990,4260.823984,5279161000
3,ARE,United Arab Emirates,1990,7161.870638,127141175000
4,ARG,Argentina,1990,247217.663647,265817210000


In [17]:
alt.Chart(merged_data.query("Year == 2018")).mark_point().encode(
    x=alt.X("GDP (constant 2015 US$):Q", scale=alt.Scale(type = 'log')),
    y=alt.Y("Food Emissions:Q", scale=alt.Scale(type = 'log'))           
).properties(
    title="Scatter plot of GDP & Food Emissions"
)

/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [18]:
chart = alt.Chart(merged_data.query("Year == 2018")).mark_point().encode(
    x=alt.X("GDP (constant 2015 US$):Q"),
    y=alt.Y("Food Emissions:Q")       
).properties(
    title="Scatter plot of GDP & Food Emissions"
)

text = alt.Chart(merged_data.query("Year == 2018")).mark_text().encode(
    x=alt.X('max(GDP (constant 2015 US$)):Q'),
    y=alt.Y('max(Food Emissions):Q'),
    text=alt.Text('Entity:O')
)

chart

/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [19]:
f_gases_df = sector[sector['Substance'] == 'F-gases']

year_columns = [col for col in f_gases_df.columns if col.startswith('Y_')]
f_gases_df['Total_Emissions'] = f_gases_df[year_columns].sum(axis=1)


top_15_nations = f_gases_df[['Name', 'Code', 'Total_Emissions']].sort_values(
    by='Total_Emissions', ascending=False
).head(15)

top_15_nations

/var/folders/7f/rvwrx89n3mb5mjys6fkrv5w00000gn/T/ipykernel_2524/1791325762.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_gases_df['Total_Emissions'] = f_gases_df[year_columns].sum(axis=1)


,Name,Code,Total_Emissions
423,China,CHN,728451.247771
531,United States,USA,706506.473354
511,Saudi Arabia,SAU,331603.490483
456,India,IND,264858.554477
524,Thailand,THA,209422.494446
439,Spain,ESP,158582.800827
443,France,FRA,156491.816977
400,United Arab Emirates,ARE,125770.902412
462,Italy,ITA,110644.639729
470,Kuwait,KWT,94511.623271


In [20]:
f_gases_df = sector[sector['Substance'] == 'F-gases']


growth_df = f_gases_df.copy()


years = [f'Y_{year}' for year in range(1990, 2019)]


for i in range(1, len(years)):
    prev_year = years[i - 1]
    current_year = years[i]
    growth_df[f'growth_{years[i]}'] = ((f_gases_df[current_year] - f_gases_df[prev_year]) / f_gases_df[prev_year]) * 100

growth_df = growth_df[['Name', 'Code'] + [f'growth_{year}' for year in years[1:]]]
growth_df.head()


year_columns = [col for col in f_gases_df.columns if col.startswith('Y_')]
f_gases_df['Total_Emissions'] = f_gases_df[year_columns].sum(axis=1)
top_15_nations = f_gases_df[['Name', 'Code', 'Total_Emissions']].sort_values(
    by='Total_Emissions', ascending=False
).head(10)

top_15_growth_df = growth_df[growth_df['Name'].isin(['China',
 'United States',
 'India',
 'Saudi Arabia',
 'Thailand',
 'United Arab Emirates',
 'Kuwait',
 'Indonesia',
 'Argentina',
 'Egypt'])]



growth_long_df = top_15_growth_df.melt(id_vars=['Name', 'Code'], 
                                       var_name='Year', 
                                       value_name='Growth Rate')
growth_long_df['Year'] = growth_long_df['Year'].str.extract('(\d+)').astype(int)

growth_long_df = growth_long_df[growth_long_df['Year'] > 1996]


line_chart = alt.Chart(growth_long_df).mark_line(color='purple').encode(
    x=alt.X('Year:O', title=None, axis=alt.Axis(labelExpr="datum.value % 2 === 0 ? datum.value : ''")),
    y='Growth Rate:Q',
    color=alt.Color(legend=None),
    strokeWidth=alt.value(4)
)

rule = alt.Chart(growth_long_df).mark_rule(strokeDash=[5, 5], color='black').encode(
    y=alt.datum(0)  # Set y to 0 for the rule
)

combined_chart = alt.layer(line_chart, rule).properties(
    width=160,  # Width of each facet
    height=70,  # Height of each facet
    title='Yearly Growth in F-gases Emissions for Top 15 Nations'
)

faceted_chart = combined_chart.facet(
    alt.Column('Name:N', header=alt.Header(labelFontSize=20), title=None),
    columns = 5,
    title=alt.TitleParams(
        text='Growth Rates of F-gases in Top 10 Emitting Nations',
        subtitle='The highest producing nations for F-gases have stabilized their growth',
        fontSize=22,
        font='Arial',
        color='Black',
        subtitleFontSize=16,
        subtitleFontStyle='italic',
        subtitleColor='darkgray'
    ),
    
)

faceted_chart


/var/folders/7f/rvwrx89n3mb5mjys6fkrv5w00000gn/T/ipykernel_2524/1808051150.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_gases_df['Total_Emissions'] = f_gases_df[year_columns].sum(axis=1)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.FacetChart(...)

In [21]:
top_15_nations.head()

f_gases_df = sector[sector['Substance'] == 'F-gases']

f_gases_df.head()


growth_long_df = f_gases_df.melt(id_vars=['Name', 'Code', 'dev_country', 'C_group_IM24_sh', 'Substance', 'FOOD_system_stage'], 
                                       var_name='Year', 
                                       value_name='Emissions')



growth_long_df['Year'] = pd.to_datetime(growth_long_df['Year'].str.replace("Y_", ""), format='%Y')
growth_long_df.head()

,Name,Code,dev_country,C_group_IM24_sh,Substance,FOOD_system_stage,Year,Emissions
0,Afghanistan,AFG,Developing,India +,F-gases,Retail,1990-01-01,0.166369
1,Angola,AGO,Developing,Southern_Africa,F-gases,Retail,1990-01-01,0.113032
2,Albania,ALB,Industrialized,Central Europe,F-gases,Retail,1990-01-01,0.042034
3,United Arab Emirates,ARE,Developing,Middle_East,F-gases,Retail,1990-01-01,3.935275
4,Argentina,ARG,Developing,Rest South America,F-gases,Retail,1990-01-01,2.830389


In [22]:

growth_long_df['Year'] = pd.to_datetime(growth_long_df['Year']).dt.year
growth_long_df1 = growth_long_df
growth_2018 = growth_long_df[growth_long_df['Year'] == 2018]

top_10 = growth_2018.nlargest(10, 'Emissions')

growth_2018['group'] = growth_2018.apply(
    lambda row: 'Top 10 Countries' if row['Name'] in top_10['Name'].values else 'Rest 185 Countries', axis=1
)
growth_2018.head()

/var/folders/7f/rvwrx89n3mb5mjys6fkrv5w00000gn/T/ipykernel_2524/741863431.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  growth_2018['group'] = growth_2018.apply(


,Name,Code,dev_country,C_group_IM24_sh,Substance,FOOD_system_stage,Year,Emissions,group
3948,Afghanistan,AFG,Developing,India +,F-gases,Retail,2018,521.739268,Rest 185 Countries
3949,Angola,AGO,Developing,Southern_Africa,F-gases,Retail,2018,354.472541,Rest 185 Countries
3950,Albania,ALB,Industrialized,Central Europe,F-gases,Retail,2018,131.819477,Rest 185 Countries
3951,United Arab Emirates,ARE,Developing,Middle_East,F-gases,Retail,2018,12341.183016,Top 10 Countries
3952,Argentina,ARG,Developing,Rest South America,F-gases,Retail,2018,8876.213938,Top 10 Countries


In [23]:
grouped_data = growth_2018.groupby('group')['Emissions'].sum().reset_index()




base = alt.Chart(grouped_data).encode(
    theta=alt.Theta(field='Emissions', type='quantitative'),
    color=alt.Color(
        field='group', 
        type='nominal',
        scale=alt.Scale(domain=['Top 10 Countries', 'Rest 185 Countries'], range=['#75d7ca', '#3b9b9a'])).legend(
        title ='Classification of Countries', labelFontSize = 15
        ).legend(None)
).properties(
    title=alt.TitleParams(
        text='F-gases Emissions Inequality',
        subtitle='10 largest emit more than 60%',
        fontSize=18,
        font='Arial',
        color='Black',
        subtitleFontSize=14,
        subtitleFontStyle='italic',
        subtitleColor='darkgray'
    ),
        width = 250, 
        height = 150
).configure_title(
    anchor='start'
)
pie_chart = base.mark_arc(outerRadius=100)


pie_chart 



/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [24]:
df_2018 = df_population[df_population['Year'] == 2018]

merged_df = pd.merge(df_2018, top_10, on='Code', how='inner')

total_population = merged_df['Population (historical)'].sum()
merged_df['PopulationRelative'] = (merged_df['Population (historical)'] / total_population) * 100

merged_df = merged_df.sort_values(by='Emissions', ascending=False)

sort_order = merged_df["Name"].tolist()

base = alt.Chart(merged_df).encode(y='Name:N')

emissions_bar = base.mark_bar(color='skyblue').encode(
    x=alt.X('Emissions:Q', title='Emissions'),
    y=alt.Y(
        'Name', 
        sort=sort_order, 
        title=None, 
        axis=alt.Axis(labelFontSize=18, titleFontSize=16, labelAlign='right'),
    )
)



population_points = base.mark_tick(
    color='purple',
    thickness=4,
    size=46 * 1.1,  
).encode(
    x=alt.X('Population (historical):Q', title='Population')
)

chart = alt.layer(emissions_bar, population_points).resolve_scale(
    x='independent'
).properties(
    width=600,
    height=400,
    title=''
)



chart

/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [25]:
sort_order

['China',
 'United States',
 'India',
 'Saudi Arabia',
 'Thailand',
 'United Arab Emirates',
 'Kuwait',
 'Indonesia',
 'Argentina',
 'Egypt']

In [26]:
top_10_sorted = top_10.sort_values(by='Emissions', ascending=False)
top_10_sorted['Name'] = top_10_sorted['Name'].replace({'United Arab Emirates': 'U.A.E.'})

top_10_sorted.head()

color_mapping = {
    'Developing': '#a0a6e9',  
    'Industrialized': '#283677'  
}


chart = alt.Chart(top_10_sorted).mark_bar(color='skyblue').encode(
    y=alt.Y(
        'Name', 
        sort='-x', 
        title=None, 
        axis=alt.Axis(labelFontSize=10, titleFontSize=16, labelAlign='right') 
    ),
    x=alt.X(
        'Emissions', 
        title='Emissions (Kilotons)', 
        axis=alt.Axis(labelFontSize=10, titleFontSize=10)
    ),
    color=alt.Color('dev_country',
                   scale=alt.Scale(domain=['Developing', 'Industrialized'],
                              range=['#a0a6e9', '#283677']), 
                   legend=alt.Legend(
            title="Development             Status",
            labelFontSize=12 
        ))
).properties(
    title=alt.TitleParams(
        text='Top 10 emitters of F-gases',
        subtitle='9 of the Top 10 are Developing Nations',
        fontSize=18,
        font='Arial',
        color='Black',
        subtitleFontSize=16,
        subtitleFontStyle='italic',
        subtitleColor='darkgray'
    ),
        width = 250, 
        height = 160
).configure_title(
    anchor='start'
)

chart


/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [27]:
top_10_sorted.head()

,Name,Code,dev_country,C_group_IM24_sh,Substance,FOOD_system_stage,Year,Emissions
3974,China,CHN,Developing,China +,F-gases,Retail,2018,95637.324421
4082,United States,USA,Industrialized,USA,F-gases,Retail,2018,61567.965616
4007,India,IND,Developing,India +,F-gases,Retail,2018,34671.028187
4062,Saudi Arabia,SAU,Developing,Middle_East,F-gases,Retail,2018,32538.363690
4075,Thailand,THA,Developing,Southeastern Asia,F-gases,Retail,2018,20549.437764


In [28]:
# Plot with Altair
countries_to_plot = [
    'China', 'United States', 'India', 'Saudi Arabia',
    'Thailand', 'United Arab Emirates', 'Kuwait',
    'Indonesia', 'Argentina', 'Egypt'
]

filtered_df = growth_long_df[growth_long_df['Name'].isin(countries_to_plot)]

# Create the line chart
chart = alt.Chart(filtered_df).mark_line().encode(
    x=alt.X('Year:T', title='Year'),
    y=alt.Y('Emissions', title='Emissions'),
    color=alt.Color('Name', title='Country', legend=None)
).properties(
    width=800,
    height=400,
    title='Yearly Growth in F-gases Emissions for Selected Nations'
)

label = chart.encode(
    x='max(Year):T',
    y=alt.Y('Emissions:Q').aggregate(argmax='Year'),
    text='Name'
)

text = label.mark_text(align='left', dx=10)

chart + text

/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [29]:
merge2 = pd.merge(df_population, growth_long_df1, on=['Code', 'Year'], how='inner')
merge2['Emissions'] = merge2['Emissions'] * 1000 * 1000
merge2['Population (historical)'] = merge2['Population (historical)'] 
merge2['Emissions Per 100'] = merge2['Emissions']/merge2['Population (historical)']


merge2.head(10)


,Entity,Code,Year,Population (historical),Name,dev_country,C_group_IM24_sh,Substance,FOOD_system_stage,Emissions,Emissions Per 100
0,Afghanistan,AFG,1990,12045664,Afghanistan,Developing,India +,F-gases,Retail,1.663688e+05,0.013812
1,Afghanistan,AFG,1991,12238879,Afghanistan,Developing,India +,F-gases,Retail,6.708702e+05,0.054815
2,Afghanistan,AFG,1992,13278982,Afghanistan,Developing,India +,F-gases,Retail,8.926528e+05,0.067223
3,Afghanistan,AFG,1993,14943174,Afghanistan,Developing,India +,F-gases,Retail,1.569702e+06,0.105045
4,Afghanistan,AFG,1994,16250799,Afghanistan,Developing,India +,F-gases,Retail,4.922841e+06,0.302929
5,Afghanistan,AFG,1995,17065836,Afghanistan,Developing,India +,F-gases,Retail,9.104374e+06,0.533485
6,Afghanistan,AFG,1996,17763265,Afghanistan,Developing,India +,F-gases,Retail,1.474404e+07,0.830030
7,Afghanistan,AFG,1997,18452100,Afghanistan,Developing,India +,F-gases,Retail,2.059639e+07,1.116209
8,Afghanistan,AFG,1998,19159996,Afghanistan,Developing,India +,F-gases,Retail,2.802533e+07,1.462700
9,Afghanistan,AFG,1999,19887791,Afghanistan,Developing,India +,F-gases,Retail,4.181810e+07,2.102702


In [30]:
df_grouped = merge2.groupby(['Year', 'dev_country']).agg({'Emissions Per 100': 'mean'}).reset_index()

line_plot = alt.Chart(df_grouped).mark_line(point=alt.OverlayMarkDef(filled=False, fill="white"), strokeWidth = 5).encode(
    x=alt.X('Year:O',title='Year'),
    y=alt.Y('Emissions Per 100:Q', title='Emissions (Kg) Per Capita'),
    color=alt.Color('dev_country',
                   scale=alt.Scale(domain=['Developing', 'Industrialized'],
                              range=['#a0a6e9', '#283677']), 
                   legend=alt.Legend(
            title="",
            labelFontSize=16 
        ))
).properties(
    title=alt.TitleParams(
        text='F-gases by Country Type',
        subtitle='Avg. Per Capita emissions of Developing Nations are bigger and diverging',
        fontSize=22,
        font='Arial',
        color='Black',
        subtitleFontSize=16,
        subtitleFontStyle='italic',
        subtitleColor='darkgray'
    ),
        width = 700, 
        height = 250
).configure_title(
    anchor='start'
)


line_plot.display()

/opt/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)